In [ ]:
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import numpy as np
import matplotlib.animation as animation
from scipy.integrate import solve_ivp


# Van der Pol方程式の定義
def van_der_pol(t, z, mu):
    x, y = z
    dxdt = y
    dydt = mu * (1 - x**2) * y - x
    return [dxdt, dydt]


# パラメータ設定
t_span = (0, 20)
t_eval = np.linspace(*t_span, 1000)
initial_conditions = [1.0, 0.0]

# tkinterウィンドウの作成
root = tk.Tk()
root.title("Van der Pol Equation Animation")
root.geometry("800x700")

# MatplotlibのFigureを用意
fig = Figure()
ax = fig.add_subplot(111)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim(-2, 2)
ax.set_ylim(-1.5, 1.5)
(line,) = ax.plot([], [], lw=2, label="Van der Pol Equation Trajectory")

# パラメータmuの範囲
mu_values = np.linspace(0.1, 2.0, 200)

# タイトル表示用のテキスト
title_text = ax.text(0.5, 1.05, "", transform=ax.transAxes, ha="center")


# アニメーションの初期化
def init():
    line.set_data([], [])
    title_text.set_text("")
    return line, title_text


# アニメーションのフレーム更新
def update(frame):
    mu = mu_values[frame]
    sol = solve_ivp(van_der_pol, t_span, initial_conditions, args=(mu,), t_eval=t_eval)
    line.set_data(sol.y[0], sol.y[1])
    title_text.set_text(f"mu = {mu:.2f}")

    # ベクトル場を計算
    x_range = np.linspace(-2, 2, 20)
    y_range = np.linspace(-1.5, 1.5, 20)
    x, y = np.meshgrid(x_range, y_range)
    dxdt = y
    dydt = mu * (1 - x**2) * y - x

    # ベクトル場の矢印を表示
    ax.clear()
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_xlim(-2, 2)
    ax.set_ylim(-1.5, 1.5)
    ax.quiver(x, y, dxdt, dydt, scale=20, color="gray", pivot="middle")
    ax.plot(sol.y[0], sol.y[1], lw=2, label="Van der Pol Equation Trajectory")
    ax.set_title(f"mu = {mu:.2f}")

    return line, title_text


# tkinterウィンドウ上にMatplotlibのFigureを表示
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

# ツールバーの追加
toolbar = NavigationToolbar2Tk(canvas, root)
toolbar.update()
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

# アニメーションの作成
ani = animation.FuncAnimation(
    fig, update, frames=len(mu_values), init_func=init, blit=True
)

# GIFファイルとして保存
ani.save("van_der_pol_animation.gif", writer="pillow", fps=60)

root.mainloop()